## 深層生成モデルではモデルの設計=目的関数の定義
- 深層生成モデルでは，いずれのモデルも最適化するための目的関数を明示的に設定する
    - 自己回帰モデル・フローベースモデル: Kullback-Leiblerダイバージェンス(対数尤度)
    - VAE: 周辺対数尤度の下界
    - GAN: Jensen-Shannonダイバージェンス(ただし目的関数自身の更新も必要(=敵対的学習))
- 推論，確率変数の表現の正則化なども，全て目的関数として追加する
<img src='../tutorial_figs/vae_loss.png'>
   
    - 深層生成モデルではモデルの設計=目的関数の定義
    - 従来の生成モデルとは異なり，サンプリングによる推論等は行わない
- 確率分布を受け取って目的関数を定義できる枠組みが必要
    - LossAPI  
<img src='../tutorial_figs/PixyzAPI.png'>

## 確率分布を受け取って目的関数を定義する
- Loss API document: https://pixyz.readthedocs.io/en/latest/losses.html#

ここではDistribution APIで定義した確率分布を受け取り目的関数を定義するまでの流れを確認する  
目的関数を定義する際には以下の項目が必要となる
1. 尤度計算をする
1. 確率分布の距離を計算する
1. 期待値を計算する
1. データ分布を考慮した計算(mean, sum)  

VAEのLossではそれぞれの項目は以下のように対応
<img src='../tutorial_figs/vae_loss_API.png'>

### Lossの計算
Loss API はlossの計算を行う際入力に確率変数を必要とします(`input_var`).
確率変数が与えられて初めてLossのあたいは計算されます.  

```python
p = DistributionAPI()
# define the objective function receiving distribution
loss = LossAPI(p)
# the value of loss is calculated when input_var is feeded
loss_value = loss.eval({'input_var': input_data})
```

In [1]:
from __future__ import print_function
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np

torch.manual_seed(1)

In [2]:
# Pixyz module
from pixyz.distributions import Normal
from pixyz.utils import print_latex

### 尤度計算を行う
ある観測値$x_1$, ...., $x_N$が得られた際，xが従うと仮定した確率分布pの尤もらしさを計算します  
ここではxは平均0, 分散1の正規分布に従うのではないかと仮定します  
$p(x) = \cal N(\mu=0, \sigma^2=1)$

In [3]:
# 確率分布pを定義
x_dim = 5
p_nor_x = Normal(var=['x'], loc=torch.tensor(0.), scale=torch.tensor(1.), features_shape=[x_dim])
print(p_nor_x)
print_latex(p_nor_x)

Distribution:
  p(x)
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([5])
    (loc): torch.Size([1, 5])
    (scale): torch.Size([1, 5])
  )


<IPython.core.display.Math object>

In [4]:
# xを観測
observed_x_num = 100
observed_x = torch.randn(observed_x_num, x_dim)
print(observed_x.shape)

torch.Size([100, 5])


対数尤度は以下のように計算されます  
$L=\sum_{i=1}^{100} \log p\left(x_{i}\right)$  
PixyzではLogProbを使用することで簡単に計算できます  
LogProbの引数にPixyz Distributionで定義した確率分布を格納し  
観測値をLogProb.eval()で渡すことで計算が行われます  
Pixyz document: https://docs.pixyz.io/en/latest/losses.html#probability-density-function

In [5]:
from pixyz.losses import LogProb
# LogProbの引数にPixyz Distributionで定義した確率分布を格納
log_likelihood_x = LogProb(p_nor_x)
print_latex(log_likelihood_x)

<IPython.core.display.Math object>

In [6]:
# 観測値それぞれに対しての尤度が計算される
print(log_likelihood_x.eval({'x': observed_x}))
# observed_x_num = 100
print('observed_x_num: ', len(log_likelihood_x.eval({'x': observed_x})))

tensor([ -7.5539,  -6.8545,  -6.4024,  -5.8851,  -6.1517,  -8.3702,  -6.7028,
         -5.0395,  -7.4346,  -7.1497,  -5.7594,  -7.3006, -11.9857,  -5.8238,
         -6.7561,  -5.7640,  -6.2382,  -4.9060,  -6.1076,  -8.2535,  -7.8250,
         -7.1956,  -7.6949,  -5.2324, -11.5860,  -8.1068,  -7.1763,  -8.3332,
        -11.4631,  -6.6297,  -6.1200, -12.2358,  -5.3402,  -7.1465,  -7.5106,
         -7.0829,  -6.6300,  -6.1832,  -7.2049, -10.8676,  -6.8674,  -5.8339,
         -9.1939,  -7.5965,  -8.7743,  -7.3492,  -5.2578, -10.3097,  -6.5646,
         -4.8807,  -5.9738,  -6.2394, -10.3945,  -9.1760,  -9.2957,  -5.5627,
         -7.1047,  -6.4066,  -6.8100,  -6.0878,  -6.8835,  -7.9132,  -5.0738,
         -8.8378,  -6.2286,  -5.8401,  -5.9691,  -5.6857,  -7.6903,  -6.4982,
         -7.1259,  -8.7953, -10.5572,  -5.9161,  -7.0649,  -6.1292,  -6.0871,
         -7.2513,  -7.2517,  -7.1378,  -6.4228,  -5.5728,  -5.6155,  -5.1962,
         -8.3940,  -7.8178,  -9.8129,  -6.1119,  -5.0492,  -8.98

log_likelihood_x.eval({'x': observed_x})には  
$\log p(x_{1})$, $\log p(x_{2})$, ...., $\log p(x_{100})$  
の計算結果が格納されている  
log_likelihood_x.eval({'x': observed_x})[i] = $\log p(x_{i})$

次に各要素を合計し
$L=\sum_{i=1}^{100} \log p\left(x_{i}\right)$を計算する  

In [7]:
# 値を合計し対数尤度を計算する
print('対数尤度の計算結果:', log_likelihood_x.eval({'x': observed_x}).sum())

対数尤度の計算結果: tensor(-715.5875)


以上のようにpixyz.lossesのLogProbを用いることで対数尤度が簡単に計算できることを確認しました  
また，定義した確率分布からp.log_prob().eval()でも同様に計算が行えます

In [8]:
print('LogProb()')
print(LogProb(p_nor_x).eval({'x': observed_x}).sum())
print('.log_prob()')
print(p_nor_x.log_prob().eval({'x': observed_x}).sum())

LogProb()
tensor(-715.5875)
.log_prob()
tensor(-715.5875)


For more Loss API related to probability density function:  
https://docs.pixyz.io/en/latest/losses.html#probability-density-function

### 確率分布の距離を計算する
生成モデルの学習では真の分布(データ分布)$p_{data}(x)$と近いモデル分布(生成モデル)$p_{\theta}(x)$を考え，適切な$\theta$を求めるために分布間の距離を測ることがある 

VAE系ではKullback-Leiblerダイバージェンス, GAN系ではJensen-Shannonダイバージェンスといったように，確率分布間の距離を計算する  
分布間距離の計算もPixyz Loss APIを用いれば簡単に行うことができる  
Pixyz document:  
https://docs.pixyz.io/en/latest/losses.html#statistical-distance  
https://pixyz.readthedocs.io/en/latest/losses.html#adversarial-statistical-distance

ここでは例として平均0, 分散1の正規分布pと平均5, 分散0.1の正規分布qとのKullback-Leiblerダイバージェンスを計算する  
$p(x) = \cal N(\mu=0, \sigma^2=1)$  
$q(x) = \cal N(\mu=5, \sigma^2=0.1)$  
$KL(q(x) || p(x))$

In [9]:
# 確率分布の定義
x_dim = 10
# p 
p_nor_x = Normal(var=['x'], loc=torch.tensor(0.), scale=torch.tensor(1.), features_shape=[x_dim])
print_latex(p_nor_x)

<IPython.core.display.Math object>

In [10]:
# q
q_nor_x = Normal(var=['x'], loc=torch.tensor(5.), scale=torch.tensor(0.1), features_shape=[x_dim], name='q')
print_latex(q_nor_x)

<IPython.core.display.Math object>

Kullback-Leiblerダイバージェンスを計算はpixyz.lossesのKullbackLeiblerを用いる  
KullbackLeibler()の引数に距離を測りたい分布を格納し   
.eval()で計算が行われる  
Pixyz document: https://docs.pixyz.io/en/latest/losses.html#kullbackleibler  

In [11]:
from pixyz.losses import KullbackLeibler

kl_q_p = KullbackLeibler(q_nor_x, p_nor_x)
print_latex(kl_q_p)

<IPython.core.display.Math object>

In [12]:
# .eval で計算を行う
kl_q_p.eval()

tensor([143.0759])

For more Loss API related to statistical distance: 
https://docs.pixyz.io/en/latest/losses.html#statistical-distance  
https://docs.pixyz.io/en/latest/losses.html#adversarial-statistical-distance  

### 期待値を計算する
何らかの関数について確率分布で重み付けして積分を行うのが期待値計算であるが
Pixyzでは潜在変数のように, input_varとして与えられない変数がある場合その変数が従う確率分布で潰
期待値の計算もLoss APIを用いれば簡単に計算できる  
Pixyz document:  
https://docs.pixyz.io/en/latest/losses.html#expected-value

ここでは例として  
$q(z|x) = \cal N(\mu=x, \sigma^2=1)$  
$p(x|z) = \cal N(\mu=z, \sigma^2=1)$  
といった二つの確率分布q(z|x), p(x|z)を考え  
$\mathbb{E}_{q(z|x)} \left[\log p(x|z) \right]$を計算する

In [13]:
# 確率分布の定義
from pixyz.distributions import Normal

q_nor_z__x = Normal(loc="x", scale=torch.tensor(1.), var=["z"], cond_var=["x"],
           features_shape=[10], name='q') # q(z|x)
p_nor_x__z = Normal(loc="z", scale=torch.tensor(1.), var=["x"], cond_var=["z"],
                    features_shape=[10]) # p(x|z)

In [14]:
# p(x|z)の対数尤度をとる
from pixyz.losses import LogProb

p_log_likelihood = LogProb(p_nor_x__z)
print_latex(p_log_likelihood)

<IPython.core.display.Math object>

期待値の計算はpixyz.lossesのExpectationを用いる  
Expectation()の引数にはp, fがあり  
期待値をとる対象の関数がfで, その関数の確率変数が従う確率分布のpで重み付けを行う  
.eval()で計算が行われる  
Pixyz document: https://docs.pixyz.io/en/latest/losses.html#expected-value

In [15]:
from pixyz.losses import Expectation as E

E_q_logprob_p = E(q_nor_z__x, LogProb(p_nor_x__z))
print_latex(E_q_logprob_p)

<IPython.core.display.Math object>

In [16]:
sample_x = torch.randn(2, 10)
E_q_logprob_p.eval({'x': sample_x})

tensor([-10.7006, -11.9861])

For more details about Expectatoin API:  
https://docs.pixyz.io/en/latest/losses.html#expected-value

### データ分布を考慮した計算(mean, sum)
本来ならxについて期待値をとる必要があるが，データ分布は実際に与えられないためbatch方向について平均や合計といった計算を行う  
合計や平均といった計算もLoss APIでは簡単に行うことができる  
ここではobserved_xを訓練データとして尤度計算を行いそのmeanを計算する
$p(x) = \cal N(\mu=0, \sigma^2=1)$  
$\frac{1}{N} \sum_{i=1}^N\left[\log p\left(x^{(i)}\right)\right]$

In [17]:
# xを観測
observed_x_num = 100
x_dim = 5
observed_x = torch.randn(observed_x_num, x_dim)
print(observed_x.shape)

torch.Size([100, 5])


In [18]:
# 確率分布pを定義
p_nor_x = Normal(var=['x'], loc=torch.tensor(0.), scale=torch.tensor(1.), features_shape=[x_dim])
print(p_nor_x)
print_latex(p_nor_x)

Distribution:
  p(x)
Network architecture:
  Normal(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=[], input_var=[], features_shape=torch.Size([5])
    (loc): torch.Size([1, 5])
    (scale): torch.Size([1, 5])
  )


<IPython.core.display.Math object>

合計や平均といった計算はLoss.mean()やLoss.sum()とすることで容易に行える

In [19]:
from pixyz.losses import LogProb
# meanを計算する
mean_log_likelihood_x = LogProb(p_nor_x).mean() # .mean()
print_latex(mean_log_likelihood_x)

<IPython.core.display.Math object>

In [20]:
mean_log_likelihood_x.eval({'x': observed_x})

tensor(-7.1973)

### Lossの組み合わせ
PixyzではLoss同士の四則演算ができる  
例として以下のLossをLoss同士の組み合わせで表現する  
$\frac{1}{N} \sum_{i=1}^{N}\left[\mathbb{E}_{q\left(z | x^{(i)}\right)}\left[\log p\left(x^{(i)} | z\right)\right]-K L\left(q\left(z | x^{(i)}\right) \| p(z)\right)\right]$

In [21]:
# 確率分布の定義
from pixyz.distributions import Normal

# p(x|z)
p_nor_x__z = Normal(loc="z", scale=torch.tensor(1.), var=["x"], cond_var=["z"],
                    features_shape=[10])

# p(z)
p_nor_z = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.), var=["z"],
                    features_shape=[10])

# q(z|x)
q_nor_z__x = Normal(loc="x", scale=torch.tensor(1.), var=["z"], cond_var=["x"],
           features_shape=[10], name='q')

In [22]:
# Lossの定義
from pixyz.losses import LogProb
from pixyz.losses import Expectation as E
from pixyz.losses import KullbackLeibler

# 対数尤度
logprob_p_x__z = LogProb(p_nor_x__z)# input_var: x, z

# 期待値E
E_q_z__x_logprob_p__z = E(q_nor_z__x, logprob_p_x__z)# input_car: x(z is not needed because of Expectation)

# KLダイバージェンス
KL_q_z__x_p_z = KullbackLeibler(q_nor_z__x, p_nor_z)

# Lossの引き算
total_loss = E_q_z__x_logprob_p__z - KL_q_z__x_p_z# input_var: x(E_q_z__x_logprob_p__z needs x as input_var)

# Lossのmean
total_loss = total_loss.mean()

# Lossの確認
print_latex(total_loss)

<IPython.core.display.Math object>

In [23]:
# Lossの計算
# xを観測
observed_x_num = 100
x_dim = 10
observed_x = torch.randn(observed_x_num, x_dim)

# 観測したxのLossを計算
total_loss.eval({'x': observed_x})

tensor(-18.9965)

以上のようにPixyz Loss API同士の四則演算で柔軟にLossが定義でき，数式から実装までが直感的に行えることが確認できた

### Loss API(ELBO)
Pixyz Loss APIでは以下のようなLossについても実装がある

周辺尤度下界 ELBO: https://docs.pixyz.io/en/latest/losses.html#lower-bound

### Next Tutorial
ModelAPITutorial.ipynb